In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 7.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from datetime import datetime, timedelta
# import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np

1

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving YouTube Comments _ all_final.xlsx to YouTube Comments _ all_final.xlsx


In [ ]:
data = pd.read_excel('YouTube Comments _ all_final.xlsx', engine='openpyxl')

# Step 3: Display basic info and the first few rows of the DataFrame
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221571 entries, 0 to 221570
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   url                  221571 non-null  object 
 1   comment_user         221361 non-null  object 
 2   content              221571 non-null  object 
 3   comment_time         221571 non-null  object 
 4   dislike_count        0 non-null       float64
 5   reply_count          106126 non-null  float64
 6   reply_user           105945 non-null  object 
 7   reply_content        106126 non-null  object 
 8   reply_comment_time   106126 non-null  object 
 9   reply_like_count     106126 non-null  object 
 10  reply_dislike_count  0 non-null       float64
 11  post_comment_count   221571 non-null  object 
 12  CrawlTime            221571 non-null  object 
 13  Channel              221571 non-null  object 
dtypes: float64(3), object(11)
memory usage: 23.7+ MB


,url,comment_user,content,comment_time,dislike_count,reply_count,reply_user,reply_content,reply_comment_time,reply_like_count,reply_dislike_count,post_comment_count,CrawlTime,Channel
0,https://www.youtube.com/watch?v=eGPCopBNieM,@JConlon96,Harris 2024 🇺🇸,18 hours ago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,486,"Wed, 06 Nov 2024 05:47:46 GMT",Harris
1,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,13 hours ago,NaN,11.0,@Angelo-fo8de,Get ready to start crying when you see Trumps ...,13 hours ago,6,NaN,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris
2,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,13 hours ago,NaN,11.0,@mfrank50,Gonna be a long night for you 😂,12 hours ago,2,NaN,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris
3,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,13 hours ago,NaN,11.0,@SuperSonic68,"@Angelo-fo8de Crying with laughter, yes. Smal...",12 hours ago,18,NaN,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris
4,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,13 hours ago,NaN,11.0,@younglee4879,@Angelo-fo8de you will find the victory of VP ...,12 hours ago,7,NaN,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris


## Data Preprocessing

In [ ]:

# Convert 'comment_time' and 'reply_comment_time' to datetime
def parse_time(time_str):
    # If the input is already a Timestamp, return it as is
    if isinstance(time_str, pd.Timestamp):
        return time_str

    # Handle NaN values
    if pd.isna(time_str):
        return np.nan

    # Current datetime
    now = datetime.now()

    # Check for different time formats
    if "hours ago" in time_str:
        hours = int(time_str.split(" ")[0])
        return now - timedelta(hours=hours)
    elif "days ago" in time_str:
        days = int(time_str.split(" ")[0])
        return now - timedelta(days=days)
    elif "weeks ago" in time_str:
        weeks = int(time_str.split(" ")[0])
        return now - timedelta(weeks=weeks)
    elif "months ago" in time_str:
        months = int(time_str.split(" ")[0])
        return now - timedelta(days=months * 30)  # Approximate each month as 30 days
    else:
        # For any other format, attempt to parse directly as a datetime
        return pd.to_datetime(time_str, errors='coerce')

# Apply the updated function
data['comment_time'] = data['comment_time'].apply(parse_time)
data['reply_comment_time'] = data['reply_comment_time'].apply(parse_time)


In [ ]:
# Display the first few rows to check the changes
data[['comment_time', 'reply_comment_time']].head()

,comment_time,reply_comment_time
0,2024-11-07 09:54:23.951044,NaT
1,2024-11-07 14:54:23.951061,2024-11-07 14:58:39.499215
2,2024-11-07 14:54:23.951065,2024-11-07 15:58:39.499236
3,2024-11-07 14:54:23.951068,2024-11-07 15:58:39.499240
4,2024-11-07 14:54:23.951071,2024-11-07 15:58:39.499242


round time

In [ ]:
# Round 'comment_time' and 'reply_comment_time' to the nearest minute
data['comment_time'] = data['comment_time'].dt.round('min')
data['reply_comment_time'] = data['reply_comment_time'].dt.round('min')

# Display the first few rows to check the changes
data[['comment_time', 'reply_comment_time']].head()



,comment_time,reply_comment_time
0,2024-11-07 09:54:00,NaT
1,2024-11-07 14:54:00,2024-11-07 14:59:00
2,2024-11-07 14:54:00,2024-11-07 15:59:00
3,2024-11-07 14:54:00,2024-11-07 15:59:00
4,2024-11-07 14:54:00,2024-11-07 15:59:00


# Text cleaning

In [ ]:
# Text cleaning function
def clean_text(text):
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Remove mentions and URLs
    text = re.sub(r'@\w+|https?://\S+', '', text)
    # Remove special characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    return text

data['cleaned_content'] = data['content'].apply(clean_text)
data['cleaned_reply_content'] = data['reply_content'].apply(lambda x: clean_text(x) if pd.notnull(x) else x)


In [ ]:
# Tokenization, stopwords removal, and lemmatization
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def preprocess_text(text):
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

data['processed_content'] = data['cleaned_content'].apply(preprocess_text)
data['processed_reply_content'] = data['cleaned_reply_content'].apply(lambda x: preprocess_text(x) if pd.notnull(x) else x)


In [ ]:
# Fill NaN values in numerical columns with 0 (if appropriate)
data[['dislike_count', 'reply_count', 'reply_like_count', 'reply_dislike_count', 'post_comment_count']] = \
    data[['dislike_count', 'reply_count', 'reply_like_count', 'reply_dislike_count', 'post_comment_count']].fillna(0)


In [ ]:

# Display the first few rows of the processed data
data.head()

,url,comment_user,content,comment_time,dislike_count,reply_count,reply_user,reply_content,reply_comment_time,reply_like_count,reply_dislike_count,post_comment_count,CrawlTime,Channel,cleaned_content,cleaned_reply_content,processed_content,processed_reply_content
0,https://www.youtube.com/watch?v=eGPCopBNieM,@JConlon96,Harris 2024 🇺🇸,2024-11-07 09:54:00,0.0,0.0,NaN,NaN,NaT,0,0.0,486,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,harris,NaN,harris,NaN
1,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@Angelo-fo8de,Get ready to start crying when you see Trumps ...,2024-11-07 14:59:00,6,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,get ready to start crying when you see trumps ...,im nauseously optimistic morning let america,get ready start cry see trump number
2,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@mfrank50,Gonna be a long night for you 😂,2024-11-07 15:59:00,2,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,gonna be a long night for you,im nauseously optimistic morning let america,gonna long night
3,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@SuperSonic68,"@Angelo-fo8de Crying with laughter, yes. Smal...",2024-11-07 15:59:00,18,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode crying with laughter yes small crowd siz...,im nauseously optimistic morning let america,fode cry laughter yes small crowd size small v...
4,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@younglee4879,@Angelo-fo8de you will find the victory of VP ...,2024-11-07 15:59:00,7,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode you will find the victory of vp harris,im nauseously optimistic morning let america,fode find victory vp harris


## mapping urls with titles



In [ ]:
from google.colab import files

# This will prompt you to upload the file
uploaded = files.upload()

Saving urls final.xlsx to urls final.xlsx


In [ ]:
titles_df = pd.read_excel("urls final.xlsx")

In [ ]:
# Merge the main dataset with the URL-title mapping based on the 'url' column
data = data.merge(titles_df, on='url', how='left')

# Display the first few rows to verify the merge
data.head()


,url,comment_user,content,comment_time,dislike_count,reply_count,reply_user,reply_content,reply_comment_time,reply_like_count,reply_dislike_count,post_comment_count,CrawlTime,Channel,cleaned_content,cleaned_reply_content,processed_content,processed_reply_content,title,Unnamed: 2
0,https://www.youtube.com/watch?v=eGPCopBNieM,@JConlon96,Harris 2024 🇺🇸,2024-11-07 09:54:00,0.0,0.0,NaN,NaN,NaT,0,0.0,486,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,harris,NaN,harris,NaN,Big Things | Harris-Walz 2024,Oprah Winfrey Welcomes First-Time Voters To A ...
1,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@Angelo-fo8de,Get ready to start crying when you see Trumps ...,2024-11-07 14:59:00,6,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,get ready to start crying when you see trumps ...,im nauseously optimistic morning let america,get ready start cry see trump number,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"
2,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@mfrank50,Gonna be a long night for you 😂,2024-11-07 15:59:00,2,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,gonna be a long night for you,im nauseously optimistic morning let america,gonna long night,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"
3,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@SuperSonic68,"@Angelo-fo8de Crying with laughter, yes. Smal...",2024-11-07 15:59:00,18,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode crying with laughter yes small crowd siz...,im nauseously optimistic morning let america,fode cry laughter yes small crowd size small v...,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"
4,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@younglee4879,@Angelo-fo8de you will find the victory of VP ...,2024-11-07 15:59:00,7,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode you will find the victory of vp harris,im nauseously optimistic morning let america,fode find victory vp harris,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"


sanity check - there should be no unmatched urls!

In [ ]:
# Count the number of rows where 'title' is NaN (i.e., no match found)
missing_titles_count = data['title'].isna().sum()

print(f"Number of URLs without a matching title: {missing_titles_count}")


Number of URLs without a matching title: 0


Exporting and saving the cleanded dataset

In [ ]:
# Save the entire merged dataset to a CSV file
data.to_csv("merged_dataset.csv", index=False)

# Optionally, download the file if you're using Google Colab
from google.colab import files
files.download("merged_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Now upload the transcits

In [ ]:
from google.colab import files

# This will prompt you to upload the file
uploaded = files.upload()

Saving Transcripts final.xlsx to Transcripts final.xlsx


Code to Combine Transcripts by URL

In [ ]:
# Load the dataset
transcripts = pd.read_excel("Transcripts final.xlsx")  # Adjust filename if necessary


In [ ]:
transcripts.head()

,Video_URL,Title,Youtuber,Post_date,Time,Text
8091,https://www.google.com/sorry/index?continue=ht...,NaN,NaN,NaN,NaN,NaN
3932,https://www.youtube.com/watch?v=-Kwbd-Duzmg,NaN,Kamala Harris,"Oct 17, 2024",0:00,when I was five I began getting sexually
3933,https://www.youtube.com/watch?v=-Kwbd-Duzmg,NaN,Kamala Harris,"Oct 17, 2024",0:02,abused by my stepfather and he got me
3934,https://www.youtube.com/watch?v=-Kwbd-Duzmg,NaN,Kamala Harris,"Oct 17, 2024",0:05,pregnant when I was
3935,https://www.youtube.com/watch?v=-Kwbd-Duzmg,NaN,Kamala Harris,"Oct 17, 2024",0:07,[Music]


In [ ]:
# Sort by Video_URL and Time to ensure order
transcripts = transcripts.sort_values(by=['Video_URL', 'Time'])

In [ ]:
transcripts = transcripts.drop(columns=['Title'])

In [ ]:
print(transcripts[[]].isna().sum())

Series([], dtype: float64)


In [ ]:
# Fill NaN values in the Text column with an empty string
transcripts['Text'] = transcripts['Text'].fillna('')

In [ ]:
# Strip leading/trailing whitespace from string columns
transcripts['Video_URL'] = transcripts['Video_URL'].str.strip()
transcripts['Youtuber'] = transcripts['Youtuber'].str.strip()
transcripts['Post_date'] = transcripts['Post_date'].str.strip()

In [ ]:
combined_transcripts = transcripts.groupby(['Video_URL', 'Youtuber', 'Post_date'])['Text'] \
                                   .apply(lambda x: ' '.join(x)) \
                                   .reset_index()

In [ ]:
combined_transcripts = combined_transcripts.rename(columns={'Text': 'Full_Transcript'})

In [ ]:
print(combined_transcripts.head())

                                     Video_URL        Youtuber     Post_date  \
0  https://www.youtube.com/watch?v=-Kwbd-Duzmg   Kamala Harris  Oct 17, 2024   
1  https://www.youtube.com/watch?v=-XINcrcvrMU   Kamala Harris   Nov 1, 2024   
2  https://www.youtube.com/watch?v=-j7X6mjrJ3M   Kamala Harris  Oct 11, 2024   
3  https://www.youtube.com/watch?v=-vEE44TNMqE   Kamala Harris   Oct 6, 2024   
4  https://www.youtube.com/watch?v=0RLhu9Ngchc  Donald J Trump  Jul 21, 2024   

                                     Full_Transcript  
0  when I was five I began getting sexually abuse...  
1  you me if you want to these expensive is red b...  
2  so my question for you is what are your plans ...  
3  we fight like hell and if you don't fight like...  
4  kamla was in on it she covered up Joe's obviou...  


export transcripts

In [ ]:
# Save the entire merged dataset to a CSV file
combined_transcripts.to_csv("combined_transcipts.csv", index=False)

# Optionally, download the file if you're using Google Colab
from google.colab import files
files.download("combined_transcipts.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# This will prompt you to upload the file
uploaded = files.upload()

Saving merged_dataset.csv to merged_dataset.csv


Handling the comment reply structure (pleas eload dataset again up so you dont have to rerun the whole code)

In [ ]:
data = pd.read_csv("merged_dataset.csv")

<ipython-input-53-d197b7d667fb>:1: DtypeWarning: Columns (9,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("merged_dataset.csv")


In [ ]:
data.head()

,url,comment_user,content,comment_time,dislike_count,reply_count,reply_user,reply_content,reply_comment_time,reply_like_count,reply_dislike_count,post_comment_count,CrawlTime,Channel,cleaned_content,cleaned_reply_content,processed_content,processed_reply_content,title,Unnamed: 2
0,https://www.youtube.com/watch?v=eGPCopBNieM,@JConlon96,Harris 2024 🇺🇸,2024-11-07 09:54:00,0.0,0.0,NaN,NaN,NaN,0,0.0,486,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,harris,NaN,harris,NaN,Big Things | Harris-Walz 2024,Oprah Winfrey Welcomes First-Time Voters To A ...
1,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@Angelo-fo8de,Get ready to start crying when you see Trumps ...,2024-11-07 14:59:00,6,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,get ready to start crying when you see trumps ...,im nauseously optimistic morning let america,get ready start cry see trump number,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"
2,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@mfrank50,Gonna be a long night for you 😂,2024-11-07 15:59:00,2,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,gonna be a long night for you,im nauseously optimistic morning let america,gonna long night,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"
3,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@SuperSonic68,"@Angelo-fo8de Crying with laughter, yes. Smal...",2024-11-07 15:59:00,18,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode crying with laughter yes small crowd siz...,im nauseously optimistic morning let america,fode cry laughter yes small crowd size small v...,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"
4,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@younglee4879,@Angelo-fo8de you will find the victory of VP ...,2024-11-07 15:59:00,7,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode you will find the victory of vp harris,im nauseously optimistic morning let america,fode find victory vp harris,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024"


In [ ]:
data['comment_id'] = data.groupby('content').ngroup() + 1


In [ ]:
data.head()

,url,comment_user,content,comment_time,dislike_count,reply_count,reply_user,reply_content,reply_comment_time,reply_like_count,...,post_comment_count,CrawlTime,Channel,cleaned_content,cleaned_reply_content,processed_content,processed_reply_content,title,Unnamed: 2,comment_id
0,https://www.youtube.com/watch?v=eGPCopBNieM,@JConlon96,Harris 2024 🇺🇸,2024-11-07 09:54:00,0.0,0.0,NaN,NaN,NaN,0,...,486,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,harris,NaN,harris,NaN,Big Things | Harris-Walz 2024,Oprah Winfrey Welcomes First-Time Voters To A ...,25769
1,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@Angelo-fo8de,Get ready to start crying when you see Trumps ...,2024-11-07 14:59:00,6,...,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,get ready to start crying when you see trumps ...,im nauseously optimistic morning let america,get ready start cry see trump number,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024",43889
2,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@mfrank50,Gonna be a long night for you 😂,2024-11-07 15:59:00,2,...,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,gonna be a long night for you,im nauseously optimistic morning let america,gonna long night,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024",43889
3,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@SuperSonic68,"@Angelo-fo8de Crying with laughter, yes. Smal...",2024-11-07 15:59:00,18,...,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode crying with laughter yes small crowd siz...,im nauseously optimistic morning let america,fode cry laughter yes small crowd size small v...,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024",43889
4,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@younglee4879,@Angelo-fo8de you will find the victory of VP ...,2024-11-07 15:59:00,7,...,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode you will find the victory of vp harris,im nauseously optimistic morning let america,fode find victory vp harris,Girl Dad | Harris-Walz 2024,"64,000 :60 | Harris-Walz 2024",43889


In [ ]:
data = data.drop(columns=['Unnamed: 2'])

# Verify that the column is removed
data.head()

,url,comment_user,content,comment_time,dislike_count,reply_count,reply_user,reply_content,reply_comment_time,reply_like_count,reply_dislike_count,post_comment_count,CrawlTime,Channel,cleaned_content,cleaned_reply_content,processed_content,processed_reply_content,title,comment_id
0,https://www.youtube.com/watch?v=eGPCopBNieM,@JConlon96,Harris 2024 🇺🇸,2024-11-07 09:54:00,0.0,0.0,NaN,NaN,NaN,0,0.0,486,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,harris,NaN,harris,NaN,Big Things | Harris-Walz 2024,25769
1,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@Angelo-fo8de,Get ready to start crying when you see Trumps ...,2024-11-07 14:59:00,6,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,get ready to start crying when you see trumps ...,im nauseously optimistic morning let america,get ready start cry see trump number,Girl Dad | Harris-Walz 2024,43889
2,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@mfrank50,Gonna be a long night for you 😂,2024-11-07 15:59:00,2,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,gonna be a long night for you,im nauseously optimistic morning let america,gonna long night,Girl Dad | Harris-Walz 2024,43889
3,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@SuperSonic68,"@Angelo-fo8de Crying with laughter, yes. Smal...",2024-11-07 15:59:00,18,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode crying with laughter yes small crowd siz...,im nauseously optimistic morning let america,fode cry laughter yes small crowd size small v...,Girl Dad | Harris-Walz 2024,43889
4,https://www.youtube.com/watch?v=2x72eSfZ7Ao,@BookMama72,I’m nauseously optimistic this morning! Let’s ...,2024-11-07 14:54:00,0.0,11.0,@younglee4879,@Angelo-fo8de you will find the victory of VP ...,2024-11-07 15:59:00,7,0.0,480,"Wed, 06 Nov 2024 05:47:46 GMT",Harris,im nauseously optimistic this morning lets do ...,fode you will find the victory of vp harris,im nauseously optimistic morning let america,fode find victory vp harris,Girl Dad | Harris-Walz 2024,43889


lnaguage of the comments and replies

Du:
cool. Do you think it makes sense to merch the transcripts with the other dataset that includes the comments per video? Or dou you think the dataset would than be too  large and unhandy for the tasks that I want to do? which is clustering the comments and build a classifier in the end.  I think it could be cool to analyse the context in which the comments were made , like for example if the video content ttranscripts wee rather emotional, angry, hopeful, provocong... ot wahtever kind of classes we could have for the videos